In [5]:
#em vez do wget do video, utilizei esta forma para importar o texto, de uma maneira mais pratica

import requests

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()  # Lê todo o conteúdo do arquivo

print("Download concluído!")


Download concluído!


In [6]:
print("length of dataset in characters: ", len(text)) #qtd de caracteres

length of dataset in characters:  1115394


In [7]:
print(text[:1000]) #1000 primeiros caracteres

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
chars = sorted(list(set(text))) # contem todos os caracteres únicos do texto(set), organizados em uma lista ordenada
vocab_size = len(chars) #recebe a qtd de caracteres unicos
print("".join(chars)) #junta os caracteres em uma unica string
print(vocab_size) #imprime a qtd caracteres unicos


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
#basicamente, o encode pega uma string e transoforma em numeros inteiros e o decode realiza o contrario, transforma
#numeors inteiros em strings

stoi = { ch:i for i,ch in enumerate(chars)} #mapeia cada caractere unico em um indice
itos = { i:ch for i,ch in enumerate(chars)} #mapeia cada indice para um caractere unico
encode = lambda s: [stoi[c] for c in s] #recebe um caractere e converte para o seu respectivo indice
decode = lambda l: "".join([itos[i] for i in l]) #recebe um indice e convercete para o seu respectivo caractere

print(encode("hii there")) #apenas imprime os indices do caracteres anteriormente mapeados
print(decode(encode("hii there"))) #imprime os caracteres dos indices anteriormente mapeados   

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
#ira codificar os 1000 caracteres para indices

import torch  
data = torch.tensor(encode(text), dtype =torch.long) #ira guardar na variavel um tensor PyTorch a partir da lista de índices
print(data.shape, data.dtype) #imprimir o shape e o tipo dos indices
print(data[:1000]) #imprimir os 1000 caracteres traduzidos para indices respecitivamente

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [11]:
n = int (0.9*len(data)) #pega o 90% de data (visto no bloco anterior)
train_data = data[:n] #utiliza estes 90% para treinammento
val_data = data[n:] #utiliza o resto para validacao

In [12]:
block_size = 8 #cada bloco de dados tera 8 elementos
train_data[:block_size+1] #os dados de treinamento ira receber 8 + 1, ou seja, 9 elementos, utilizado para "previsao" de um dado posterior

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size] #seleciona os 8 primeiros elementos
y = train_data[1:block_size+1] #seleciona os seguintes 8 elementos (bloco a ser previsto a partir da entrada)
for t in range(block_size):
    context = x[:t+1] #percorre o bloco de codigo ate o seu ultimo, adcionando um dado a cada iteracao
    target = y[t] # proximo caractere a ser previsto apos o contexto.
    print(f"quando a entrada ser {context} o alvo: {target}")

quando a entrada ser tensor([18]) o alvo: 47
quando a entrada ser tensor([18, 47]) o alvo: 56
quando a entrada ser tensor([18, 47, 56]) o alvo: 57
quando a entrada ser tensor([18, 47, 56, 57]) o alvo: 58
quando a entrada ser tensor([18, 47, 56, 57, 58]) o alvo: 1
quando a entrada ser tensor([18, 47, 56, 57, 58,  1]) o alvo: 15
quando a entrada ser tensor([18, 47, 56, 57, 58,  1, 15]) o alvo: 47
quando a entrada ser tensor([18, 47, 56, 57, 58,  1, 15, 47]) o alvo: 58


In [14]:
torch.manual_seed(1337) #define a semente para o gerador de numeros aleatorios
batch_size = 4 # sequencias independentes que serao executadas paralelamentes
block_size = 8 # numero maximo para cada bloco do contexto para previsoes

def get_batch(split): 
    #caso o split for train, usamos os dados de treinamento, caso contrario, usamos os dados de valicao
    data = train_data if split == 'train' else val_data
    #gera 4 indices aleatorios dentro do intervalo de len(data) - block_size (o que garente que seja divisivel por block_size)
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    #responsavel por pegar e empilhar varias subsequencias com um intervalo do block_size, que neste caso e 8
    x = torch.stack([data[i:i+block_size] for i in ix])
    #o y pegar e empilhar o subsequente de x, ou seja, os proximos caracteres apos cada subsequencia de x
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train') # volta os dados de entrada e saida para o conjunto train
print('inputs:')
print(xb.shape) #[batch_size, block_size]
print(xb) #conteudo das entradas (subsequencias anteriores)
print('targets:')
print(yb.shape) #[batch_size, block_size]
print(yb) #conteudo dos alvos (proximos caracteres apos a subsequencia de xb)

print('----')

#preve o proximo caractere pelo contexto crescente anterior, ou seja, para cada "conjunto", 
#o modelo tentara prever qual sera o proximo, com o context crescendo, enquanto o target(alvo), sera o proximo numero
for b in range(batch_size): 
    for t in range(block_size): 
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [15]:
print(xb) #imprime o transformador (dado de entrada)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        #cria uma tabela com os embedding(numeros representando palavras), para mapear uma distribuicoes de logits
        #assim conseguindo a previsao de porcentagem de palavras poderia completar a frase
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    #responsavel pelo processamento da entrada e pela geracao da saida do modelo.
    def forward(self, idx, targets=None):

        # o indice e passado pela camada de embeddings, retornando um logits para cada token de sequencia
        logits = self.token_embedding_table(idx) # gera tensor logits(B(batch_size),T(tamanho da sequencia),C(vocab_size))

        if targets is None: #caso nao tenha palavras corretas para comparar, apenas continua
            loss = None
        else:
            # "achatamos" o shape, multipliando o B e T para, o resultado sendo o total de previsoes e o C representando as possibilidades
            B, T, C = logits.shape
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            #usando a cross_entrpoty para saber o quao errada sao as previsoes, comparadando com as palavras corretas(targets)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # idx e o (B, T) array(sequencia inicial das palavras), e o maximo de tokens que o modelo deve gerar
    def generate(self, idx, max_new_tokens):
            for _ in range(max_new_tokens):
                # faz uma previsao baseada no texto atual, retornando logits
                logits, loss = self(idx)
                # pega a ultima palavra do contexto para prever a proxima, ignorando as anteriores
                logits = logits[:, -1, :] # becomes (B, C)
                # aplica o softmax para transformas cada probabildiade dos logits em porcentagem entre 0 e 1
                probs = F.softmax(logits, dim=-1) # (B, C)
                # escolha um logit aleatorio (quanto maior porcentagem, mais chance de ser escolhida)
                idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
                # adciona no final da sequencia, assim o modelo tera um contexto melhor para a proxima rodada
                idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            return idx #retorna a sequencia completa

m = BigramLanguageModel(vocab_size) #criacao de um modelo m
logits, loss = m(xb, yb) #chamando o modelo para treinar/testar
print(logits.shape)
print(loss)

#comeca com o texto no indice 0 e gera 100 novas palavras, e o decode transforma os numeros em texto novamente
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [17]:
# criando um otimizacor para o modelo m, onde a taxa de aprendizado e lr=1e-3(aproximadamente 0.001)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [18]:
batch_size = 32
for steps in range(10000): # o treinamento ira ocorrer em 100 epocas

    # pegar um conjutno de exemplos na base de dados de train, retornando uma matriz, no caso o xb(entrada), yb(target)
    xb, yb = get_batch('train')

    # calcula os logits e a funcao de perda(loss), mostrando o quao errado esta a previsao
    logits, loss = m(xb, yb)
    #zera os gradientes anteriores a fim de evitar possiveis conflitos
    optimizer.zero_grad(set_to_none=True)
    # mostra o quao errado o modelo estava em relacao a previsao, com este feedback, conseguimos ajustar os parametros
    loss.backward()
    #com base no resultado anterior, aplica os ajustes nos pesos
    optimizer.step()

#imprime o valor de perda final
print(loss.item())

2.5727508068084717


In [19]:
#comecando com uma sequencia de comprimento 1 (um único token) para 1 exemplo de batch, ferando 500 novos tokens
#converte para uma lista, e o encode transforma em frase
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


In [20]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) #matriz de pesos normalizados
a = a / torch.sum(a, 1, keepdim=True) 
b = torch.randint(0,10,(3,2)).float() #valores a serem agradados
c = a @ b #realiza a multiplicacao de matrizes, onde cada linha de c e uma media ponderada das linhas de b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [21]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels, ou seja, tridimensional, fixando para o numeros sempre serem os mesmo
x = torch.randn(B,T,C) #define os tamanho de btc
x.shape #numeros aleatorios normais com media 0, desvio padrao 1

torch.Size([4, 8, 2])

In [22]:
# We want x[b,t] = mean_{i<=t} x[b,i], usando for que nao seria a maneira mais eficiente
xbow = torch.zeros((B,T,C)) #cria um tensor cheio de zeros
for b in range(B): #percorre as frases
    for t in range(T): #percorre todas as palavras dentro da frase 
        xprev = x[b,:t+1] # (t,C) contem todas as palavras anteriores e a atual ate o tempo t
        xbow[b,t] = torch.mean(xprev, 0) #calcula a media das polavras anteriores e substitui a palavra atual por essa media calculada


In [23]:
x[0] #os 8 elementos 

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [24]:
xbow[0] #os elementos agora calculado com a sua media

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [25]:
# vmultiplicacao de matrizes, realizando a media acumulada de uma forma mais eficiente
wei = torch.tril(torch.ones(T, T)) #cria uma matriz triangular inferior preenchidos com 1, e o triangular supeior com 0
wei = wei / wei.sum(1, keepdim=True) #cada linha de 1 vai ser normalizada
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C) #para cada tempo, pegar os valores anteriores em x e multiplciar pela matriz wei
torch.allclose(xbow, xbow2) #verifica se o calculo realizado agora e o com looping sao quase identicos

False

In [26]:
xbow2[0] 

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [27]:
print(torch.max(torch.abs(xbow - xbow2)))  #a diferenca nao e tao grande de um para o outro, sendo praticamente 0


tensor(3.2363e-08)


In [28]:
# version 3: usando softmax
tril = torch.tril(torch.ones(T, T)) #cria a mesma matriz triangular inferiores com 1
wei = torch.zeros((T,T)) #matriz wei cheia de 0
wei = wei.masked_fill(tril == 0, float('-inf')) #substitue os valores da diagonal para cima(os zeros), e substitui por -inf, fazendo o softmax ignorar
wei = F.softmax(wei, dim=-1) #aplica o softmax na ultima dimensao, convertendo os valores em probabildiades
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [29]:
# version 4: faz com que tenha um mecanismo, fazendo com que cada palavra fique de olho nas palavras anteriores
torch.manual_seed(1337)
B,T,C = 4,8,32 # BTC
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False) #representa a identidade de cada palavra
query = nn.Linear(C, head_size, bias=False) #representa o q esta sendo procurado em outras palavras
value = nn.Linear(C, head_size, bias=False) #representa as informacoes a serem usadas na saida
k = key(x)   # (B, T, 16) #chave das palavras, diminuindo a dimensionalidade para 16, tornaod a computacao mais eficiente
q = query(x) # (B, T, 16) #consulta das palavras
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T), ficando (4,8,8), armazenando o quanto cada palavra deve prestar atencao nas outras

tril = torch.tril(torch.ones(T, T)) #matriz triangular inferior
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) #substitui os valores a cima da diagonal por -inf, bloqueando o futuro
wei = F.softmax(wei, dim=-1) #softmax, probabilidade entre 0 e 1

v = value(x) #transforma os valores de x em valores v, com forma (B,T,16)
out = wei @ v #multiplixa os pesos da atencao(wei) pelos valores
#out = wei @ x, onde cada palavra tem as informacoes mais relevantes de cada palavra anterior

out.shape

torch.Size([4, 8, 16])

In [30]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [31]:
k = torch.randn(B,T,head_size) #matriz de chaves
q = torch.randn(B,T,head_size) #matriz de consultas
wei = q @ k.transpose(-2, -1) * head_size**-0.5 #calculo das matriz,gerando um mapa onde cada linha representa o quanto uma palavra tem a ver com a outra
#e depois e aplicado um scaling, para normalizar os valores e evitar numeros mto grande

In [32]:
k.var() #variancia dos elementos de k

tensor(1.0449)

In [33]:
q.var() #calcular a variancia dos elementos de q

tensor(1.0700)

In [34]:
wei.var() #calcular a variancia dos elementos de wei

tensor(1.0918)

In [35]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1) # probabilidade de cada numero

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [36]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # multipliocando por 8, o maior valor fica muito maior que o restante
#assim, se aproxima de uma distribuicao one-hot, quase toda probabilidade vai para o valor mais alto

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [37]:
class LayerNorm1d: # camada de normalziacao, serve para ajustar os dados

  def __init__(self, dim, eps=1e-5, momentum=0.1): 
    self.eps = eps #salvando o valor de eps
    self.gamma = torch.ones(dim) #tensor gamma com o tamanho de dim preenchido por 1, usado para ajustar amplitude
    self.beta = torch.zeros(dim) #tensor beta com o tamanho de dim, usado para ajustar deslocamento

  def __call__(self, x): #onde a normalizacao ocorrera
    xmean = x.mean(1, keepdim=True) # calcula a media de cada linha dos dados, e a media ao longo das colunas de cada linha
    xvar = x.var(1, keepdim=True) #caclula a variancia de cada linha dos dados, e a variancia ao longo das colunas de cada linha
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normaliza os dados, para a media ficar 0 e a variancia 1
    self.out = self.gamma * xhat + self.beta #aplicando os ajustes de escala (gamma) e deslocamento(beta) nos dados normalizados
    return self.out #valor normlizado ajustado

  def parameters(self): #usado para obter os parametros da camada, usados para ajustar os dados, como anteriormente explciados
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100) #100 caracteristeicas para normalizar, pq a entrada tem 100 dimensoes
x = torch.randn(32, 100) # 32 exemplos e cada exemplo tem 100 caracteristicas, com numeros aleatorios
x = module(x) #passando os dados de x para module
x.shape

torch.Size([32, 100])

In [38]:
x[:,0].mean(), x[:,0].std() # calcula a media e o desvio padrao dos 32 valres da primerira caracteristicas 

(tensor(0.1469), tensor(0.8803))

In [39]:
x[0,:].mean(), x[0,:].std() #calcula a media e o desvio padrao dos 100 valores do primeiro exemplo

(tensor(-9.5367e-09), tensor(1.0000))

In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hiperparametros
batch_size = 16 # quantas sequencias independentes serao processadas em paralelo?
block_size = 32 # qual o comprimento maximo do contexto para previsoes?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# baixar o arquivo de texto
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# aqui estao todos os caracteres unicos que ocorrem neste texto
chars = sorted(list(set(text)))
vocab_size = len(chars)
# criar um mapeamento de caracteres para inteiros
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # codificador: recebe uma string e retorna uma lista de inteiros
decode = lambda l: ''.join([itos[i] for i in l]) # decodificador: recebe uma lista de inteiros e retorna uma string

# dividir em conjunto de treino e validacao
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # primeiros 90% serao treino, o resto validacao
train_data = data[:n]
val_data = data[n:]

# carregamento de dados
def get_batch(split):
    # gerar um pequeno lote de dados de entradas x e alvos y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ uma cabeca de autoatencao """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # calcular os pesos de atencao
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # realizar a agregacao ponderada dos valores
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ varias cabecas de autoatencao em paralelo """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ uma camada linear simples seguida de uma nao linearidade """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ bloco do transformer: comunicacao seguida de computacao """

    def __init__(self, n_embd, n_head):
        # n_embd: dimensao do embedding, n_head: numero de cabecas de atencao
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# modelo bigrama super simples
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # cada token acessa diretamente os logits do proximo token a partir de uma tabela de busca
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # camada final de normalizacao
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx e targets sao ambos tensores (B,T) de inteiros
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx e um array (B, T) de indices no contexto atual
        for _ in range(max_new_tokens):
            # recorta idx para os ultimos block_size tokens
            idx_cond = idx[:, -block_size:]
            # obtem as previsoes
            logits, loss = self(idx_cond)
            # foca apenas no ultimo passo de tempo
            logits = logits[:, -1, :] # fica (B, C)
            # aplica softmax para obter probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # amostra da distribuicao
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # adiciona o indice amostrado a sequencia
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# imprime o numero de parametros do modelo
print(sum(p.numel() for p in m.parameters())/1e6, 'M parametros')

# cria um otimizador do pytorch
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # de tempos em tempos avalia a perda nos conjuntos de treino e validacao
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # amostra um lote de dados
    xb, yb = get_batch('train')

    # avalia a perda
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# gera texto a partir do modelo
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5058
step 300: train loss 2.4195, val loss 2.4335
step 400: train loss 2.3507, val loss 2.3569
step 500: train loss 2.2964, val loss 2.3128
step 600: train loss 2.2407, val loss 2.2499
step 700: train loss 2.2055, val loss 2.2191
step 800: train loss 2.1640, val loss 2.1870
step 900: train loss 2.1240, val loss 2.1503
step 1000: train loss 2.1022, val loss 2.1294
step 1100: train loss 2.0695, val loss 2.1179
step 1200: train loss 2.0379, val loss 2.0796
step 1300: train loss 2.0247, val loss 2.0643
step 1400: train loss 1.9931, val loss 2.0374
step 1500: train loss 1.9707, val loss 2.0302
step 1600: train loss 1.9617, val loss 2.0471
step 1700: train loss 1.9414, val loss 2.0149
step 1800: train loss 1.9082, val loss 1.9963
step 1900: train loss 1.9080, val loss 1.9873
step 2000: train loss 1.8844, val loss 1.9958
step 2100: train loss 1.